In [46]:
from telethon import TelegramClient
import pandas as pd
import json 
from pathlib import Path
import json
import os
from dotenv import load_dotenv
import asyncio
from asyncio import TimeoutError
from pprint import pprint
print(load_dotenv())
api_id = os.getenv("TELEGRAM_API_ID")
api_hash = os.getenv("TELEGRAM_API_HASH")
phone_num = os.getenv("PHONE_NUM")
session_name = "GMGN_ATH_Group"
from telethon.errors import ChatAdminRequiredError, ChannelPrivateError
from telethon.sessions import MemorySession
from telethon.sessions import StringSession

True


In [34]:
# Add this function to save the session string
async def save_session():
    client = TelegramClient(StringSession(), api_id, api_hash)
    await client.start()
    session_string = client.session.save()
    with open('session.txt', 'w') as f:
        f.write(session_string)
    await client.disconnect()
    return session_string

# Only run this once to create the session
await save_session()

Signed in successfully as DM; remember to not break the ToS or you will risk an account ban!


'1AZWarzoBu54TBSwZO5G82HI7rcVnrU_7d2t-NgAoDmNFJ3MHfjRDwDAvGmryPSSOVoqxo8kOAf7lzwzEx3XEywdSxD3tJSHLgJDYBu4I_5uolLZcJT-RtrjyV3PG_HATa2BVOinrTfAqOAvDHN-c1qtMGrhRskklJty-dEONkkp8vwGhgLdSejO_aiVsfDIRN-OIAo57JGXKncB3_XSuYtipiTp6QLa9SNyjwmTD6igJIs0i6uCgLoqDE2oXKrxMASiVocOmLAWVNbDwZFGrANvPwHX26MuQ1DQjMwtibKyGmIvH39r5QJcYteuA8GWJX9D0-eF7U1g1sg5uMiBZKP_LOI1hG3s='

In [86]:
from telethon.tl.functions.channels import GetForumTopicsRequest
from telethon.sessions import StringSession

async def main(chat_name, limit, topic_id=None):
    # Load existing session
    try:
        with open('session.txt', 'r') as f:
            session_string = f.read()
        client = TelegramClient(StringSession(session_string), api_id, api_hash)
    except FileNotFoundError:
        print("No saved session found, using memory session")
        client = TelegramClient(MemorySession(), api_id, api_hash)
    
    try:
        print("Starting client...")
        await client.start()
        
        try:
            print(f"Attempting to connect to chat: {chat_name}")
            # Get chat info 
            chat_info = await client.get_entity(chat_name)
            print(f"Successfully connected to chat: {chat_info.title}")
            
            # Get topic title if topic_id is provided
            topic_title = None
            if topic_id:
                result = await client(GetForumTopicsRequest(
                    channel=chat_info,
                    offset_date=0,
                    offset_id=0,
                    offset_topic=0,
                    limit=100
                ))
                topic_title = next((topic.title for topic in result.topics 
                                  if hasattr(topic, 'id') and topic.id == topic_id), "Unknown Topic")
                print(f"Fetching messages from topic: {topic_title} (ID: {topic_id})")
            
            # Get all the messages from the specific topic
            print("Fetching messages...")
            messages = await client.get_messages(
                entity=chat_info, 
                limit=limit,
                reply_to=topic_id
            )
            print(f"Retrieved {len(messages)} messages")
            return {
                "messages": messages, 
                "channel": chat_info,
                "topic_title": topic_title
            }
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None
            
    finally:
        print("Disconnecting client...")
        await client.disconnect()

async def main_with_timeout(chat_name, limit, topic_id=None, timeout=120):
    try:
        result = await asyncio.wait_for(main(chat_name, limit, topic_id), timeout=timeout)
        return result
    except TimeoutError:
        print(f"Operation timed out after {timeout} seconds")
        return None

In [87]:
print('''
ID: 1115814 - Title: KOL FOMO
ID: 1115806 - Title: Smart Money FOMO
ID: 1 - Title: General
ID: 1383119 - Title: Hall of Fame
ID: 1152511 - Title: PUMP FDV Surge
ID: 1152525 - Title: Solana FDV Surge
ID: 1268284 - Title: Solana New Pool Alert
ID: 1273756 - Title: Pump King of the hill（KOTH）
ID: 1153178 - Title: Heavy Bought
ID: 1152477 - Title: DEV Burnt Alert
ID: 1272338 - Title: Pump Completed Alert
ID: 1115976 - Title: ATH Price
ID: 1436168 - Title: Pump DEV Activity
ID: 1120024 - Title: Update DEX Screener Social
ID: 1115997 - Title: Sniper New
ID: 1116022 - Title: Pump Update DEX Screener Social
ID: 1115926 - Title: CTO
ID: 1272382 - Title: Sol Lp Burn
ID: 1115948 - Title: Advertised on DEXScreener
ID: 167118 - Title: Pump DEV Activity
''')



ID: 1115814 - Title: KOL FOMO
ID: 1115806 - Title: Smart Money FOMO
ID: 1 - Title: General
ID: 1383119 - Title: Hall of Fame
ID: 1152511 - Title: PUMP FDV Surge
ID: 1152525 - Title: Solana FDV Surge
ID: 1268284 - Title: Solana New Pool Alert
ID: 1273756 - Title: Pump King of the hill（KOTH）
ID: 1153178 - Title: Heavy Bought
ID: 1152477 - Title: DEV Burnt Alert
ID: 1272338 - Title: Pump Completed Alert
ID: 1115976 - Title: ATH Price
ID: 1436168 - Title: Pump DEV Activity
ID: 1120024 - Title: Update DEX Screener Social
ID: 1115997 - Title: Sniper New
ID: 1116022 - Title: Pump Update DEX Screener Social
ID: 1115926 - Title: CTO
ID: 1272382 - Title: Sol Lp Burn
ID: 1115948 - Title: Advertised on DEXScreener
ID: 167118 - Title: Pump DEV Activity



In [94]:
# Test the connection with a smaller limit first
chat_name = "gmgnsignals"
topic_id = 1115976  # This is the ID for ATH Price topic
limit = 2

print("Starting main function...")
result = await main_with_timeout(chat_name, limit, topic_id)
if result and result["messages"]:
    print(f"\nMessages from topic: {result['topic_title']}")
    print("-" * 50)
    print("Latest message:", result["messages"][0].text)
    print("-" * 50)
else:
    print("No messages retrieved")

Starting main function...
Starting client...


Attempting to connect to chat: gmgnsignals
Successfully connected to chat: GMGN Featured Signals - SOL
Fetching messages from topic: ATH Price (ID: 1115976)
Fetching messages...
Retrieved 2 messages
Disconnecting client...

Messages from topic: ATH Price
--------------------------------------------------
Latest message: **💊ATH Price💊**

**$Ghost**(save me)
`8XDmh6RJToZi5w8aDf7ZC8f2QtxGGFUxY78aUDakpump`

📈 5m | 1h | 6h: **19.79%** | **1604.2%** | **1604.2%**
🎲 5m TXs/Vol: **396**/**$75.4K**
💡 MCP: **$103.9K**
💧 Liq: **79.01** **SOL** ($34K 🔥100%)
👥 Holder: **285**
🕒 Open: **2min** **ago**

✅ NoMint / ✅Blacklist / ✅Burnt
✅TOP 10: **23.92%**

⏳ DEV: [1.76% HOLD](https://gmgn.ai/sol/address/Cm1xQByhCegsVKMYWsZZRbgSvgSNKm2D4e8i5U7bT6L8)
👨‍🍳 DEV Burnt烧币: -

Backup BOT: [US](https://t.me/US_GMGNBOT?start=8XDmh6RJToZi5w8aDf7ZC8f2QtxGGFUxY78aUDakpump) | [01](https://t.me/GMGN_sol_bot?start=8XDmh6RJToZi5w8aDf7ZC8f2QtxGGFUxY78aUDakpump) | [02](https://t.me/GMGN_sol02_bot?start=8XDmh6RJToZi5w8aDf7

In [92]:
from telethon.tl.types import Message, Channel, ForumTopicDeleted
from telethon.tl.functions.channels import GetForumTopicsRequest
from telethon.sessions import StringSession

async def get_topics(chat_name):
    # Load existing session
    try:
        with open('session.txt', 'r') as f:
            session_string = f.read()
        client = TelegramClient(StringSession(session_string), api_id, api_hash)
    except FileNotFoundError:
        print("No saved session found, using memory session")
        client = TelegramClient(MemorySession(), api_id, api_hash)
    
    try:
        print("Starting client...")
        await client.start()
        
        try:
            print(f"Attempting to connect to chat: {chat_name}")
            chat = await client.get_entity(chat_name)
            
            if not isinstance(chat, Channel):
                print("This is not a channel/group")
                return None
                
            print("Fetching topics...")
            result = await client(GetForumTopicsRequest(
                channel=chat,
                offset_date=0,
                offset_id=0,
                offset_topic=0,
                limit=100
            ))
            
            topics = []
            for topic in result.topics:
                # Skip deleted topics
                if isinstance(topic, ForumTopicDeleted):
                    continue
                    
                topic_data = {
                    'id': topic.id,
                    'title': topic.title if hasattr(topic, 'title') else 'Untitled',
                    'top_message': topic.top_message if hasattr(topic, 'top_message') else None
                }
                topics.append(topic_data)
            
            print(f"\nFound {len(topics)} active topics:")
            for topic in topics:
                print(f"ID: {topic['id']} - Title: {topic['title']}")
            
            return topics
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None
            
    finally:
        print("Disconnecting client...")
        await client.disconnect()

# Use it like this:
chat_name = "gmgnsignals"
topics = await get_topics(chat_name)

# Save topics to file if needed
if topics:
    with open('telegram_topics.json', 'w') as f:
        json.dump(topics, f, indent=2)
    print("\nTopics saved to telegram_topics.json")

SyntaxError: incomplete input (3517003661.py, line 1)